<div style="text-align: center; line-height: 0; padding-top: 2px;">
  <img src="https://www.quantiaconsulting.com/logos/quantia_logo_orizz.png" alt="Quantia Consulting" style="width: 600px; height: 250px">
</div>

# Retrieving Weather Data

## OpenWeatherMap API

* It is possible to retrieve the current weather for any location, specified by its name or its geographical coordinates (latitute and longitude): the results are sent in JSON format.
* The API gives the **forecasting** for each location.
* Historical data can be queried, too.
* Python API Wrapper: _PyOWM_ (https://pyowm.readthedocs.io/en/latest/index.html)

## Instructions

1. Generate an API key from OpenWeatherMap (https://openweathermap.org/appid)
1. Interact with the API:
 `Extract the weather of a specific location and show the forecasting for the next week.`

In [10]:
import json
from pyowm import OWM
import pandas as pd

## Weather API Exercise

Insert API key to get access to the API.

In [2]:
credentials = json.load(open('credentials.json'))['weather']
api_key = credentials['APIKey']

In [3]:
owm = OWM(api_key)

### a) Obtain current weather for 8 cities around the world
_Berlin, Chicago, San Francisco, Mumbai, Seoul, Nairobi, London, Mexico City_

In [4]:
cities = ['Berlin', 'Chicago', 'San Francisco', 'Mumbai', 'Seoul', 'Nairobi', 'London', 'Mexico City']
countries = ['DE', 'US', 'US', 'IN', 'KR', 'KE', 'GB', 'MX']

Use registry to get location ID.

In [5]:
reg = owm.city_id_registry()

Store results in an object for further processing.

In [6]:
places = {}
i = 0
for city in cities:
    for loc in reg.locations_for(city, country=countries[i]):
        
        place_id = loc.get_ID()
        
        l = {}
        l['name'] = loc.get_name()
        l['lat'] = loc.get_lat()
        l['long'] = loc.get_lon()
        
        places[place_id] = l
    
    i += 1

In [8]:
for id_loc in places:
    print (id_loc, places[id_loc])

(3530597, {'lat': 19.428471, 'name': u'Mexico City', 'long': -99.127663})
(4887398, {'lat': 41.850029, 'name': u'Chicago', 'long': -87.650047})
(1835848, {'lat': 37.56826, 'name': u'Seoul', 'long': 126.977829})
(184745, {'lat': -1.28333, 'name': u'Nairobi', 'long': 36.816669})
(1275339, {'lat': 19.01441, 'name': u'Mumbai', 'long': 72.847939})
(2950158, {'lat': 54.033329, 'name': u'Berlin', 'long': 10.45})
(2950159, {'lat': 52.524368, 'name': u'Berlin', 'long': 13.41053})
(5391959, {'lat': 37.774929, 'name': u'San Francisco', 'long': -122.419418})
(2643743, {'lat': 51.50853, 'name': u'London', 'long': -0.12574})


### b) Obtain weather in the _surroundings_ of each city
Limit the surroundings to a maximum of 10 other cities and store results in tabular format.

In [11]:
header = ['id_location', 'name_loc', 'name_surrounding', 'lat', 'long','temperature','status','humidity','clouds', 'rain','timestamp']

In [31]:
results = pd.DataFrame(columns=header)
for id_loc in places:
    place = places[id_loc]
    surrounding_obs = owm.weather_around_coords(place['lat'], place['long'], limit=10)
    
    for obs in surrounding_obs:
        loc = obs.get_location()
        w = obs.get_weather()
        
        if len(w.get_rain().values()) > 0:
            rain = list(w.get_rain().values())[0]
        else:
            rain = 0
            
        results = results.append({'id_location': id_loc,
                                  'name_loc': place['name'],
                                  'name_surrounding': loc.get_name(),
                                  'lat': loc.get_lat(),
                                  'long': loc.get_lon(),
                                  'temperature': w.get_temperature(unit='celsius')['temp'],
                                  'status': w.get_status(),
                                  'humidity': w.get_humidity(),
                                  'clouds': w.get_clouds(),
                                  'rain': rain,
                                 'timestamp': w.get_reference_time(timeformat='iso')}, ignore_index=True)

In [32]:
results

,id_location,name_loc,name_surrounding,lat,long,temperature,status,humidity,clouds,rain,timestamp
0,3530597,Mexico City,Mexico City,19.4285,-99.1277,9.10,Clear,54,5,0,2019-02-07 08:48:00+00
1,3530597,Mexico City,Cuauhtemoc,19.4172,-99.1570,10.17,Clear,70,5,0,2019-02-07 09:46:00+00
2,3530597,Mexico City,Venustiano Carranza,19.4300,-99.0992,9.02,Clear,54,5,0,2019-02-07 08:48:00+00
3,3530597,Mexico City,Viaducto Piedad,19.4000,-99.1334,10.13,Clear,70,5,0,2019-02-07 09:46:00+00
4,3530597,Mexico City,Colonia Roma,19.4182,-99.1631,10.17,Clear,70,5,0,2019-02-07 09:46:00+00
5,3530597,Mexico City,Portales,19.4000,-99.1501,10.17,Clear,70,5,0,2019-02-07 09:46:00+00
6,3530597,Mexico City,San Pedro,19.4000,-99.1000,9.05,Clear,54,5,0,2019-02-07 08:48:00+00
7,3530597,Mexico City,Colonia Nativitas,19.3812,-99.1369,10.17,Clear,70,5,0,2019-02-07 09:46:00+00
8,3530597,Mexico City,El Gas,19.4667,-99.1667,10.19,Clear,70,5,0,2019-02-07 09:46:00+00
9,3530597,Mexico City,Colonia del Valle,19.3861,-99.1621,10.20,Clear,70,5,0,2019-02-07 09:46:00+00


##### ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) 2020 Quantia Consulting, srl. All rights reserved.